I'm trying to figure out if I can use the ContactLQR methods for BotAtlas without the robot desperately trying to leap back to the origin (since all I actually want is for it to come to a stop anywhere). 

To simplify the issue, I'm going to create a very simple 2D robot with a planar (non-revolute) base and a single foot. 

In [1]:
using Revise

In [13]:
using LCPSim
using MeshCat
using MeshCatMechanisms
using RigidBodyDynamics
using LearningMPC
using Gurobi

In [3]:
vis = Visualizer()
IJuliaCell(vis)

Listening on 127.0.0.1:7001...
zmq_url=tcp://127.0.0.1:6001
web_url=http://127.0.0.1:7001/static/


MeshCat.IJuliaCell(MeshCat.ViewerWindow(ZMQ.Context(Ptr{Void} @0x000000000494a3b0, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x0000000004b601d0, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000050a7030, 57, (1, 0), Condition(Any[]), (false, false), 0))]), ZMQ.Socket(Ptr{Void} @0x0000000004b601d0, Base.Filesystem._FDWatcher(Ptr{Void} @0x00000000050a7030, 57, (1, 0), Condition(Any[]), (false, false), 0)), "http://127.0.0.1:7001/static/", "tcp://127.0.0.1:6001", MeshCat.ZMQServer.ZMQWebSocketBridge(ip"127.0.0.1", Set{WebSockets.WebSocket}(), MeshCat.SceneTrees.SceneNode(Nullable{Array{UInt8,1}}(), Nullable{Array{UInt8,1}}(), Dict{String,MeshCat.SceneTrees.SceneNode}()), 7001, HttpServer.Server(HttpServer.HttpHandler(MeshCat.ZMQServer.handle_file_request, Base.TCPServer(RawFD(42) active), Dict{String,Function}(Pair{String,Function}("error", HttpServer.#1),Pair{String,Function}("listen", HttpServer.#3))), WebSockets.WebSocketHandler(MeshCat.ZMQServer.#5)), ZMQ.Context(Ptr{Void} @0x0000000005026180, ZMQ.Socket[ZMQ.Socket(Ptr{Void} @0x00000000049452b0, Base.Filesystem._FDWatcher(Ptr{Void} @0x0000000004b3cd00, 49, (1, 0), Condition(Any[]), (false, false), 0)), ZMQ.Socket(Ptr{Void} @0x0000000004e4d270, Base.Filesystem._FDWatcher(Ptr{Void} @0x000000000494ff40, 50, (1, 0), Condition(Any[Task (runnable) @0x00007fa366e12230]), (true, false), 0))]), ZMQ.Socket(Ptr{Void} @0x0000000004e4d270, Base.Filesystem._FDWatcher(Ptr{Void} @0x000000000494ff40, 50, (1, 0), Condition(Any[Task (runnable) @0x00007fa366e12230]), (true, false), 0)), "tcp://127.0.0.1:6001")), false)

In [11]:
function get_xstar()
    xstar = MechanismState(mechanism)
    set_configuration!(xstar, [0, 1, 0, 1])
    xstar
end
    

get_xstar (generic function with 1 method)

In [92]:
urdf = "hopper2d.urdf"
mechanism = parse_urdf(Float64, urdf)
mvis = MechanismVisualizer(mechanism, URDFVisuals(urdf), vis)
env = LCPSim.parse_contacts(mechanism, urdf)
contacts = [Point3D(default_frame(findbody(mechanism, "foot")), 0., 0., 0.)]

Q = diagm([0., 10, 1, 10, 1, 1, 1, 1])
R = diagm([0.01, 0.01, 0.01, 0.01])
Δt = 0.01
xstar = get_xstar()
lqrsol = LQRSolution(xstar, Q, R, Δt, contacts)

(::LQRSolution) (generic function with 2 methods)

In [93]:
x0 = get_xstar()
set_configuration!(x0, findjoint(mechanism, "base_x"), [0.5])
# LearningMPC.randomize!(x0, xstar, 0.0, 2.0)
results = LCPSim.simulate(x0, lqrsol, env, Δt, 200, GurobiSolver(Gurobi.Env(); OutputFlag=0));

In [94]:
playback(mvis, results, Δt)

In [95]:
lqrsol.K

4×8 Array{Float64,2}:
  0.0           0.0           0.0          …  0.0          0.0        
  0.0           0.0           0.0             0.0          0.0        
 -4.64288       9.51503e-15   4.64288         6.91054      9.85811e-16
  1.02694e-12  20.5385       -1.02694e-12     3.33632e-14  7.91969    

In [96]:
lqrsol.S

8×8 Array{Float64,2}:
  37.2104        -7.90167e-14  -37.2104       …  -2.50624      -6.71604e-15
  -2.85518e-13  192.801          2.85518e-13      9.19375e-13  11.2083     
 -37.2104         7.90167e-14   37.2104           2.50624       6.71604e-15
  -2.85518e-13  192.801          2.85518e-13      9.19375e-13  11.2083     
   2.50624       -5.38735e-15   -2.50624         -3.9678       -5.82389e-16
   6.0759e-13    11.2083        -6.0759e-13   …   2.48948e-14   4.51589    
  -2.50624        5.38735e-15    2.50624          3.9678        5.82389e-16
   6.0759e-13    11.2083        -6.0759e-13       2.48948e-14   4.51589    

In [97]:
x0 = get_xstar()
u = LearningMPC.nominal_input(x0, contacts)
Jc = LCPSim.ContactLQR.contact_jacobian(x0, contacts)
ẋ = LCPSim.ContactLQR.dynamics_with_contact_constraint(x0, u, Jc)

8-element Array{Float64,1}:
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 1.77636e-15
 0.0        
 1.77636e-15

In [98]:
A, B, c = LCPSim.ContactLQR.contact_linearize(x0, u, Jc)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.77636e-15, 0.0, 1.77636e-15])

In [99]:
A

8×8 Array{Float64,2}:
 0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [100]:
B

8×4 Array{Float64,2}:
 0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0
 0.0  0.0  -1.0  0.0
 0.0  0.0   0.0  1.0
 0.0  0.0   1.0  0.0
 0.0  0.0   0.0  1.0

In [101]:
c

8-element Array{Float64,1}:
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 1.77636e-15
 0.0        
 1.77636e-15

In [102]:
x1 = [1, 1, 0, 1, 0, 0, 0, 0]
(x1 - lqrsol.x0)' * lqrsol.Q * (x1 - lqrsol.x0)

0.0

In [103]:
K, S = LCPSim.ContactLQR.lqr(A, B, Q, R)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; -9.25569e-8 -0.00224947 … 14.8068 672.531; 1.63452e-5 20.4393 … -0.00219727 100.0], [0.0 -0.0 … -0.0 0.0; 22.8248 6.67427e5 … 6.30216e6 -1.6821e11; … ; -1.0 9.85366e-6 … 0.147722 15.951; -5.21741e5 -1.26805e10 … -1.44225e11 3.79105e15])

In [104]:
K

4×8 Array{Float64,2}:
  0.0          0.0          0.0         …     0.0     0.0           0.0  
  0.0          0.0          0.0               0.0     0.0           0.0  
 -9.25569e-8  -0.00224947  10.0            -672.531  14.8068      672.531
  1.63452e-5  20.4393      -1.71047e-5      -86.0    -0.00219727  100.0  

In [105]:
S

8×8 Array{Float64,2}:
   0.0         -0.0           -0.0         …  -0.0            0.0       
  22.8248       6.67427e5     22.8246          6.30216e6     -1.6821e11 
  -1.54307e-8  -0.000375024    1.48324         1.09573      112.121     
 -22.8248      -6.67423e5    -22.8246         -6.30216e6      1.6821e11 
  -1.0          3.23484e-5     1.33096e-9     -0.000345763    9.22574   
   5.21741e5    1.26805e10     5.21735e5   …   1.44225e11    -3.79105e15
  -1.0          9.85366e-6     0.1             0.147722      15.951     
  -5.21741e5   -1.26805e10    -5.21735e5      -1.44225e11     3.79105e15

In [112]:
x0 = get_xstar()
# set_configuration!(x0, findjoint(mechanism, "base_x"), [0.5])
LearningMPC.randomize!(x0, xstar, 0.0, 1.0)
results = LCPSim.simulate(x0, x -> -K * (Vector(x) - lqrsol.x0) + lqrsol.u0, env, Δt, 400, GurobiSolver(Gurobi.Env(); OutputFlag=0));
playback(mvis, results, Δt)

In [113]:
using LearningMPC.Models

In [208]:
robot = BoxAtlas()
xstar = nominal_state(robot)
delete!(vis)
mvis = MechanismVisualizer(robot, vis)
params = MPCParams(robot)
lqrsol = LQRSolution(robot)
contacts = [Point3D(default_frame(robot.feet[:left]), 0., 0., 0.),
         Point3D(default_frame(robot.feet[:right]), 0., 0., 0.)]

2-element Array{RigidBodyDynamics.Spatial.Point3D{StaticArrays.SArray{Tuple{3},Float64,1,3}},1}:
 Point3D in "after_core_to_lf_extension": [0.0, 0.0, 0.0]
 Point3D in "after_core_to_rf_extension": [0.0, 0.0, 0.0]

In [209]:
x0 = nominal_state(robot)
u = LearningMPC.nominal_input(x0, contacts)
Jc = LCPSim.ContactLQR.contact_jacobian(x0, contacts)
A, B, c = LCPSim.ContactLQR.contact_linearize(x0, u, Jc)
K, S = LCPSim.ContactLQR.lqr(A, B, lqrsol.Q, lqrsol.R)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.000458357 5.60744e10 … -1.67755e14 7.91881e13; -0.000458525 -5.60744e10 … 1.67755e14 -7.91881e13], [-5.73845e-22 -8.94551e-8 … 0.000157582 -0.000257261; 3.22729e-15 355.12 … -42437.0 -23625.8; … ; -2.11147e-10 -1.03307e5 … -8.15312e7 4.95228e7; -2.44722e-10 -3.5097e5 … 1.70786e8 -1.23411e8])

In [210]:
K

11×22 Array{Float64,2}:
  0.0           0.0         …      0.0     0.0          0.0       
  0.0           0.0                0.0     0.0          0.0       
  0.0           0.0                0.0     0.0          0.0       
 -0.00257033   -3.14395e11     -2459.94    9.40561e14  -4.43988e14
  0.00257041    3.14395e11      2459.93   -9.40561e14   4.43988e14
 -0.00293      -3.58364e11  …  -2803.5     1.0721e15   -5.06081e14
  0.00293       3.58364e11      2803.5    -1.0721e15    5.06081e14
  0.000749724   9.17342e10       717.839  -2.74437e14   1.29547e14
 -0.000750322  -9.17342e10      -715.274   2.74437e14  -1.29547e14
  0.000458357   5.60744e10       440.181  -1.67755e14   7.91881e13
 -0.000458525  -5.60744e10  …   -440.182   1.67755e14  -7.91881e13

In [211]:
x0 = nominal_state(robot)
# set_configuration!(x0, findjoint(mechanism, "base_x"), [0.5])
LearningMPC.randomize!(x0, xstar, 0.0, 0.5)
results = LCPSim.simulate(x0, x -> -K * (Vector(x) - lqrsol.x0) + lqrsol.u0, environment(robot), params.Δt, 400, GurobiSolver(Gurobi.Env(); OutputFlag=0));

In [212]:
playback(mvis, results, params.Δt)

In [213]:
lqrsol.S[1,:] .= 0
lqrsol.S[:,1] .= 0
lqrsol.K[:,1] .= 0

11-element SubArray{Float64,1,Array{Float64,2},Tuple{Base.Slice{Base.OneTo{Int64}},Int64},true}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [214]:
x0 = nominal_state(robot)
LearningMPC.randomize!(x0, xstar, 0.0, 0.5)
results = LCPSim.simulate(x0, lqrsol, environment(robot), params.Δt, 400, GurobiSolver(Gurobi.Env(); OutputFlag=0));

In [215]:
playback(mvis, results, params.Δt)

In [216]:
mpc_controller = MPCController(robot, (p = MPCParams(robot); p.horizon=1; p), lqrsol, [lqrsol])

(::MPCController) (generic function with 1 method)

In [249]:
x0 = nominal_state(robot)
# LearningMPC.randomize!(x0, xstar, 0.0, 2.0)
set_velocity!(x0, findjoint(robot.mechanism, "floating_base"), [-2, 0, 0])
results = LCPSim.simulate(x0, mpc_controller, environment(robot), params.Δt, 200, GurobiSolver(Gurobi.Env(); OutputFlag=0));

In [250]:
playback(mvis, results, params.Δt)